This is just a simple implementation of likelihood encoding (also known as impact coding or target coding) for the categorical features in python.

Just trying to understand how it works.

I'm pretty certain code can be improved, specially regarding the creation of the mapping and its application, but I preserved the same implementation I used in which application to the training data, and creation of the encoding map are two different steps.

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold


In [2]:
train_data = pd.read_csv('../input/train.csv')
test_data = pd.read_csv('../input/test.csv')

In [3]:
features = train_data.columns[2:]

numeric_features = []
categorical_features = []

for dtype, feature in zip(train_data.dtypes[2:], train_data.columns[2:]):
    if dtype == object:
        #print(column)
        #print(train_data[column].describe())
        categorical_features.append(feature)
    else:
        numeric_features.append(feature)
categorical_features

['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8']

In [4]:
# This way we have randomness and are able to reproduce the behaviour within this cell.
np.random.seed(13)

def impact_coding(data, feature, target='y'):
    '''
    In this implementation we get the values and the dictionary as two different steps.
    This is just because initially we were ignoring the dictionary as a result variable.
    
    In this implementation the KFolds use shuffling. If you want reproducibility the cv 
    could be moved to a parameter.
    '''
    n_folds = 20
    n_inner_folds = 10
    impact_coded = pd.Series()
    
    oof_default_mean = data[target].mean() # Gobal mean to use by default (you could further tune this)
    kf = KFold(n_splits=n_folds, shuffle=True)
    oof_mean_cv = pd.DataFrame()
    split = 0
    for infold, oof in kf.split(data[feature]):
            impact_coded_cv = pd.Series()
            kf_inner = KFold(n_splits=n_inner_folds, shuffle=True)
            inner_split = 0
            inner_oof_mean_cv = pd.DataFrame()
            oof_default_inner_mean = data.iloc[infold][target].mean()
            for infold_inner, oof_inner in kf_inner.split(data.iloc[infold]):
                # The mean to apply to the inner oof split (a 1/n_folds % based on the rest)
                oof_mean = data.iloc[infold_inner].groupby(by=feature)[target].mean()
                impact_coded_cv = impact_coded_cv.append(data.iloc[infold].apply(
                            lambda x: oof_mean[x[feature]]
                                      if x[feature] in oof_mean.index
                                      else oof_default_inner_mean
                            , axis=1))

                # Also populate mapping (this has all group -> mean for all inner CV folds)
                inner_oof_mean_cv = inner_oof_mean_cv.join(pd.DataFrame(oof_mean), rsuffix=inner_split, how='outer')
                inner_oof_mean_cv.fillna(value=oof_default_inner_mean, inplace=True)
                inner_split += 1

            # Also populate mapping
            oof_mean_cv = oof_mean_cv.join(pd.DataFrame(inner_oof_mean_cv), rsuffix=split, how='outer')
            oof_mean_cv.fillna(value=oof_default_mean, inplace=True)
            split += 1
            
            impact_coded = impact_coded.append(data.iloc[oof].apply(
                            lambda x: inner_oof_mean_cv.loc[x[feature]].mean()
                                      if x[feature] in inner_oof_mean_cv.index
                                      else oof_default_mean
                            , axis=1))

    return impact_coded, oof_mean_cv.mean(axis=1), oof_default_mean

# Apply the encoding to training and test data, and preserve the mapping
impact_coding_map = {}
for f in categorical_features:
    print("Impact coding for {}".format(f))
    train_data["impact_encoded_{}".format(f)], impact_coding_mapping, default_coding = impact_coding(train_data, f)
    impact_coding_map[f] = (impact_coding_mapping, default_coding)
    mapping, default_mean = impact_coding_map[f]
    test_data["impact_encoded_{}".format(f)] = test_data.apply(lambda x: mapping[x[f]]
                                                                         if x[f] in mapping
                                                                         else default_mean
                                                               , axis=1)

Impact coding for X0
Impact coding for X1
Impact coding for X2
Impact coding for X3
Impact coding for X4
Impact coding for X5
Impact coding for X6
Impact coding for X8


In [5]:
train_data[['y', 'X0'] + list(train_data.columns[-8:])]

,y,X0,impact_encoded_X0,impact_encoded_X1,impact_encoded_X2,impact_encoded_X3,impact_encoded_X4,impact_encoded_X5,impact_encoded_X6,impact_encoded_X8
0,130.81,k,99.531103,101.487334,104.107950,102.674595,100.767715,127.785386,101.097037,97.751981
1,88.53,k,99.569691,94.023006,95.346500,99.936023,100.764441,89.743039,99.302406,97.754279
2,76.26,az,78.048052,96.113110,83.682071,102.040561,100.764441,78.440000,101.096261,98.968275
3,80.62,az,78.046404,94.008796,83.680029,96.805025,100.767717,78.440000,99.277779,105.107536
4,78.02,az,78.043751,101.486740,83.674062,96.795811,100.767715,80.290703,101.273632,102.299600
5,92.93,t,93.837830,99.938070,98.381857,102.040942,100.764436,93.699107,103.079767,99.177454
6,128.76,al,93.660734,100.558772,98.389122,96.795789,100.764437,105.528748,103.079129,99.173594
7,91.91,o,94.762972,100.718778,103.837383,96.796199,100.764441,105.541981,101.096918,100.281919
8,108.67,w,112.906937,101.925801,103.837217,99.931662,100.767709,105.528748,101.366663,103.076597
9,126.99,j,112.324678,99.937680,105.934569,102.040746,100.767715,105.541981,97.815509,105.104869
